In [1]:
import flask
import pandas as pd
import pytesseract
from pytesseract import Output
import cv2
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from flask import request,jsonify
import cv2
import numpy as np
from flask_cors import CORS, cross_origin

app=flask.Flask(__name__)

@app.route('/', methods = ['GET', 'POST'])
@cross_origin()
def handle_request():
    print('in the upload_file!')
    if request.method == 'POST':
        text = request.get_json()
    #splitting into individual sentences
    sentences = []
    sentences.append(sent_tokenize(text["data"]))
    length = len(sentences)
    sentences = [y for x in sentences for y in x]
    #GloVe word embeddings
    word_embeddings = {}
    f = open(r'C:\Users\dhruv\Downloads\archive\glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    #text pre processing
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]"," ")
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')
    def remove_stopwords(sentence):
        new = " ".join([i for i in sentence if i not in stop_words])
        return new
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #Vector representation of sentences
    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)
    #similiarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    #Pagerank algorithm using graph
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    final = ""
    for i in range(4):
          final+=(ranked_sentences[i][1])
    return (final)
app.run(host="0.0.0.0",port=5000,debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
